In [1]:
class LazyDB(object):
    def __init__(self):
        self.exists = 5
        
    # 존재하지 않는 프로퍼티에 접근할 때 호출되는 함수 
    def __getattr__(self, name):
        value = 'Value for %s % name'
        setattr(self, name, value)
        return value

data = LazyDB()
print('Before: ', data.__dict__)
print('foo: ', data.foo)
print('After: ', data.__dict__)


Before:  {'exists': 5}
foo:  Value for %s % name
After:  {'exists': 5, 'foo': 'Value for %s % name'}


In [3]:
class LogginLazyDB(LazyDB):
    def __getattr__(self, name):
        print('Called __getattr__(%s)' % name)
        return super().__getattr__(name)

data = LogginLazyDB()
print('exists: ', data.exists) # 존재하는 데이터
print('foo: ', data.foo)
print('foo: ', data.foo) # 위 라인에서 foo에 접근했기 때문에, __getattr__이 다시 불리지 않음

exists:  5
Called __getattr__(foo)
foo:  Value for %s % name
foo:  Value for %s % name


In [4]:
class ValidatingDB(object):
    def __init__(self):
        self.exists = 5
    
    # 객체의 프로퍼티가 있든 없든 호출
    def __getattribute__(self, name): 
        print('Called __getattribute__(%s)' % name)
        try:
            return super().__getattribute__(name)
        except AttributeError:
            value = 'Value for %s' % name
            setattr(self, name, value)
            return value
        
data = ValidatingDB()
print('exists: ', data.exists)
print('foo: ', data.foo)
print('foo: ', data.foo)

Called __getattribute__(exists)
exists:  5
Called __getattribute__(foo)
foo:  Value for foo
Called __getattribute__(foo)
foo:  Value for foo


In [5]:
data = LogginLazyDB()
print('Before: ', data.__dict__)
print('foo exists', hasattr(data, 'foo')) # __getattr__ 호출
print('After: ', data.__dict__)
print('foo exists: ', hasattr(data, 'foo')) # __getattr__ 호출 안함

Before:  {'exists': 5}
Called __getattr__(foo)
foo exists True
After:  {'exists': 5, 'foo': 'Value for %s % name'}
foo exists:  True


In [6]:
data = ValidatingDB()
print('foo exists: ', hasattr(data, 'foo')) # __getattribute__ 호출
print('foo exists: ', hasattr(data, 'foo')) # __getattribute__ 호출

Called __getattribute__(foo)
foo exists:  True
Called __getattribute__(foo)
foo exists:  True


In [7]:
# lazy 하게 파이썬 객체에 값을 할당해야 한다면 __setattr__을 씁시다.

class SavingDB(object):
    def __setattr__(self, name, value):
        # 몇몇 데이터를 DB 로그로 저장함
        # ...
        super().__setattr__(name, value)
        
class LogginSavingDB(SavingDB):
    def __setattr__(self, name, value):
        print('Called __setattr__(%s, %r)' % (name, value))
        super().__setattr__(name, value)

data = LogginSavingDB()
print('Before: ', data.__dict__)
data.foo = 5
print('After: ', data.__dict__)
data.foo = 7
print('Finally: ', data.__dict__)

Before:  {}
Called __setattr__(foo, 5)
After:  {'foo': 5}
Called __setattr__(foo, 7)
Finally:  {'foo': 7}


In [8]:
"""
__getattribute__와 __setattr__ 을 사용할 때 부딪히는 문제가 있다.
객체의 속성에 접근할 때마다 (원치 않을 때도) 호출된다는 점이다.
"""

class BrokenDictionaryDB(object):
    def __init__(self, data):
        self._data = {}
    
    def __getattribute__(self, name):
        print('Called __getattrivute__(%s)' % name)
        return self._data[name]

data = BrokenDictionaryDB({'foo':3})
data.foo # __getattribute__ 메서드에서 self._data에 접근해서 __getattribute__ 메서드가 다시 실행.. 재귀로 인한 스택오버플로


Called __getattrivute__(foo)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __getattrivute__(_data)
Called __g

RecursionError: maximum recursion depth exceeded

In [10]:
class DictionaryDB(object):
    def __init__(self, data):
        self._data = data
    
    def __getattribute__(self, name):
        data_dict = super().__getattribute__('_data') # super().__getattribute__ 메서드로 인스턴스 속성 딕셔너리에서 값을 얻어온다.
        return data_dict[name]

data = DictionaryDB({'foo':3})
data.foo

3